<a href="https://colab.research.google.com/github/sanjanb/Machine-Learning-basics/blob/main/NeurakNetwork_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **What is this code doing?**
This code is building and training a **neural network** to recognize handwritten digits (0-9) from the **MNIST dataset**, which contains images of handwritten digits.

---

### **Step-by-step explanation**

#### **1. Loading the MNIST Dataset**
```python
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
```
- The MNIST dataset contains 60,000 training images and 10,000 testing images of handwritten digits (0-9).
- Each image is 28x28 pixels in grayscale.
- `transforms.ToTensor()` converts the images into PyTorch tensors (a format PyTorch can work with).
- `transforms.Normalize((0.5,), (0.5,))` adjusts the pixel values to be between -1 and 1 for better training performance.

---

#### **2. Creating Data Loaders**
```python
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
```
- A **DataLoader** helps us load the data in small batches (e.g., 64 images at a time) instead of all at once.
- `shuffle=True` means the training data is shuffled randomly before each epoch (one full pass through the dataset).
- `shuffle=False` for the test data ensures we evaluate the model on the same order every time.

---

#### **3. Defining the Neural Network**
```python
class NeuralNet(torch.nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    self.fc1 = torch.nn.Linear(28*28, 128)
    self.fc2 = torch.nn.Linear(128, 64)
    self.fc3 = torch.nn.Linear(64, 10)
  
  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
```
- This defines a simple neural network with three layers:
  - **Input Layer**: Takes a flattened version of the 28x28 image (784 pixels).
  - **Hidden Layers**: Two fully connected (dense) layers with 128 and 64 neurons, respectively. These use the **ReLU activation function** to introduce non-linearity.
  - **Output Layer**: Outputs 10 values (one for each digit: 0-9). These represent the "confidence" that the input image belongs to each digit.

---

#### **4. Creating the Model, Loss Function, and Optimizer**
```python
model = NeuralNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
```
- `model = NeuralNet()` creates an instance of the neural network.
- `criterion = torch.nn.CrossEntropyLoss()` defines the loss function, which measures how wrong the model's predictions are compared to the true labels.
- `optimizer = torch.optim.Adam(...)` sets up the optimizer, which adjusts the model's weights during training to minimize the loss.

---

#### **5. Training the Model**
```python
epochs = 10
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    optimizer.zero_grad()  # Reset gradients
    outputs = model(images)  # Forward pass (predict)
    loss = criterion(outputs, labels)  # Compute loss
    loss.backward()  # Backward pass (compute gradients)
    optimizer.step()  # Update weights
  print(f"epoch {epoch+1}, Loss:{loss.item()}")
```
- The model is trained for 10 epochs (10 full passes through the training data).
- For each batch of images:
  1. The model makes predictions (`outputs`).
  2. The loss is calculated by comparing predictions to the true labels.
  3. Gradients are computed using backpropagation (`loss.backward()`).
  4. The optimizer updates the model's weights to reduce the loss (`optimizer.step()`).
- After each epoch, the current loss is printed to track progress.

---

#### **6. Evaluating the Model**
```python
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f"Accuracy on the test set: {100 * correct / total}%")
```
- After training, the model is tested on the unseen test dataset to evaluate its performance.
- For each batch of test images:
  1. The model predicts the digit (`outputs`).
  2. The predicted digit is compared to the true label.
  3. Correct predictions are counted.
- Finally, the accuracy (percentage of correct predictions) is printed.

---

### **Summary**
1. **Dataset**: Loads handwritten digit images (MNIST).
2. **Model**: Builds a simple neural network with three layers.
3. **Training**: Trains the model using the training data and adjusts weights to minimize prediction errors.
4. **Testing**: Evaluates the model on unseen test data to measure its accuracy.

By the end, the model should be able to recognize handwritten digits with reasonable accuracy (e.g., ~95% or higher).

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define a neural network
class NeuralNet(torch.nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    self.fc1 = torch.nn.Linear(28*28, 128)
    self.fc2 = torch.nn.Linear(128, 64)
    self.fc3 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

# The issue was on this line, it should be an assignment (=), not a subtraction (-)
model = NeuralNet()

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
  print(f"epoch {epoch+1}, Loss:{loss.item()}")

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total}%")

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 52.6MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.88MB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 13.1MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.28MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

epoch 1, Loss:0.24933141469955444
epoch 2, Loss:0.0605836920440197
epoch 3, Loss:0.20769014954566956
epoch 4, Loss:0.0070379069074988365
epoch 5, Loss:0.17141886055469513
epoch 6, Loss:0.00596141442656517
epoch 7, Loss:0.03204083815217018
epoch 8, Loss:0.028609707951545715
epoch 9, Loss:0.11041945964097977
epoch 10, Loss:0.0026470590382814407
Accuracy on the test set: 96.78%


In [6]:
# Save the model
torch.save(model.state_dict(), 'mnist_model.pth')
print("Wodel saved!!")

# Load the model
loaded_model = NeuralNet()
loaded_model.load_state_dict(torch.load('mnist_model.pth')) # Correctly load the state dict
print("Model loaded!!")


# make the predictions
with torch.no_grad():
  for batch in test_loader:
    images, labels = batch
    output = loaded_model(images)
    _, predicted = torch.max(output, 1)
    print("Predictions:", predicted[:10])
    print("Actual:", labels[:10])
    break

Wodel saved!!
Model loaded!!
Predictions: tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])
Actual: tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])


<ipython-input-6-4e6b9688dc6b>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('mnist_model.pth')) # Correctly load the state dict
